In [ ]:
from utils.config import Config
from dataloader.docomo_datasource import DocomoDataSource
from models.transformer import Transformer

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib as plt

In [ ]:
#config = Config()
#ds = DocomoDataSource(config)
#ds.vocab_size

In [ ]:
units = [256, 512]
layers = [2, 4, 6]
lrs = [0.001]
configs = []
for l in layers:
    for u in units:
        for lr in lrs:
            configs.append(Config(num_layers=l, num_units=u, learning_rate=lr, log_dir='./logs/transformer/'))

In [ ]:
num_epochs = 20

In [ ]:
def run(configs, gpu_index, num_epochs=100):
    for config in configs:
        with tf.Graph().as_default():
            with tf.device('/gpu:{}'.format(gpu_index)):
                ds = DocomoDataSource(config)
                model = Transformer(config, 'transformer')

                global_step = tf.train.get_or_create_global_step()
                optimizer = tf.train.AdamOptimizer(config.learning_rate)
                train_op = optimizer.minimize(model.loss, global_step=global_step)


                with tf.name_scope('summary'):
                    loss_smr = tf.summary.scalar('loss', model.loss)
                    acc_smr = tf.summary.scalar('acc', model.accuracy)
                    merged_summary = tf.summary.merge_all()

                tf_config = tf.ConfigProto(
                    allow_soft_placement=True,
                    gpu_options=tf.GPUOptions(
                        allow_growth=True
                    )
                )
                with tf.Session(config=tf_config) as sess:
                    writer = tf.summary.FileWriter(model.config.to_log_dir() , sess.graph)
                    sess.run(tf.global_variables_initializer())
                    for epoch in range(num_epochs):
                        ds.shuffle()
                        batch_list = ds.feed_dict(model, model.config.batch_size, is_transformer=True)
                        for fd in batch_list:
                            fd[model.is_training] = True
                            _, step, loss, acc, smr = sess.run([train_op, global_step, model.loss, model.accuracy, merged_summary], feed_dict=fd)
                            #step = sess.run(global_step)
                            writer.add_summary(smr, step)
                            #if step % 100 == 0:
                            #print('step: {}, loss: {:.3f}, acc: {:.3f}'.format(step, loss, acc))
                        print('epoch {}/{} finished.'.format(epoch+1, num_epochs))

In [ ]:
run(configs, 1, num_epochs)

In [ ]:
#import threading

In [ ]:
#thread1 = threading.Thread(target=run, args=(configs[::2], 0, 100))
#thread2 = threading.Thread(target=run, args=(configs[1::2], 1, 100))
#threads = [thread1, thread2]
# for t in threads:
#     t.setDaemon(True)
#     t.start()

In [ ]:
# for t in threads:
#     t.join()